<a href="https://colab.research.google.com/github/Mushahid2521/Rice-Grain-Purity-Analysis-Using-Deep-Learning/blob/master/Rice_Grain_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rice grain Analysis Using Deep Convolutional Neural Network
In this Notebook we will see the implementation of the paper named **Rice Purity Analysis using Deep learning**. Our task was to take a scanned image of rice grains like this and convert it to like this to this.  

<img src="https://github.com/Mushahid2521/Rice-Grain-Purity-Analysis-Using-Deep-Learning/blob/master/images/Test15.jpg?raw=1" alt="Image" height="600" width="400" align="left"/> 
<img src="https://github.com/Mushahid2521/Rice-Grain-Purity-Analysis-Using-Deep-Learning/blob/master/images/predicted_img.jpg?raw=1" alt="Image" height="600" width="400" align="right"/>



## Step 1: Dataset Preparation
First of all we need to detect the Individual Rice grains. We can do this by Canny edge detection algorithm. Then we store these individual grains from the Scanned images for training.
Code below this takes the Dataset directory containing Scanned grains with corresponding grain type name as Folder name and create another directory containing trainable individual grain dataset after some processing steps. The steps are: 

1.   Apply Edge detection to full image
2.  Find the Contours of the edges to detect individual grains
3.  Create a mask to extract the grains and making the background completely black
4.  Crop the grains using  opencv ```boundingRect()``` function.
5.  Pad the cropped images to specific size (to retain aspect ratio)
6.  Save the padded images to definite directory.







In [0]:
import cv2
import imutils
import os
import numpy as np
import pading_the_grain



def prepare():

    PATH = 'DATASET'
    
    NEW_PATH = 'TRAINABLE_DATASET'
    os.makedirs(NEW_PATH)


    for type in os.listdir(PATH):
        TYPE_PATH = os.path.join(PATH,type)

        print("Processing type {}".format(type))
        
        if type in os.listdir(NEW_PATH):
          continue 

        NEW_TYPE_PATH = os.path.join(NEW_PATH,type)
        os.makedirs(NEW_TYPE_PATH)

        i = 1

        for image in os.listdir(TYPE_PATH):
            IMAGE_PATH = os.path.join(TYPE_PATH,image)

            print("Processing Image....{}".format(image))

            img = cv2.imread(IMAGE_PATH)
            img_grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

            canny_img = cv2.Canny(img_grey, 100, 100)


            cnts = cv2.findContours(canny_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)

            mask = np.ones(img.shape[:2], dtype="uint8") * 255
            cv2.drawContours(mask, cnts, -1, 0, -1)
            img[mask == 255] = 0

            for c in cnts:
                (x, y, w, h) = cv2.boundingRect(c)
                # Take only the right grains, in many cases it might remove the broken grains
                if (w*h >1600):
                    crop = img[y:y + h, x:x + w]
                    try:
                        crop = pading_the_grain.pad_image(crop, 128, 0)
                    except:
                        continue

                    # remove the images with edges only by counting the non zero pixel values
                    if ((49152 - len(crop[crop == 0])) < 2000):
                        continue

                    image_name = "{}_{}.png".format(type,i)
                    image_url = os.path.join(NEW_TYPE_PATH,image_name)

                    cv2.imwrite(image_url, crop)

                    i += 1

                   

        print(f'Got type {type}', len(os.listdir(NEW_TYPE_PATH)))


prepare()

## Step 2: Data Augmentation
The dataset contains different number of scanned images for different grain types. So we increase the amount of data and balance the data by rotating a single grains to different orientations. 

In [0]:
# Path containing individual grain dataset
PATH = 'TRAINABLE_DATASET'

# Directory to save the new dataset after augmentation 
NEW_DATA_PATH = 'ENLARGED_TRAINABLE_DATASET'
os.makedirs(NEW_DATA_PATH)

# Total number of grains we want to make for each grain type
TOTAL_GRAINS = 20000

for grain_type in os.listdir(PATH):
  p = os.path.join(PATH, grain_type)
  
  print(f'Working with {grain_type}')
  grains = os.listdir(p)
  length = len(grains)
  
  if length>TOTAL_GRAINS:
    continue
  
  more_needed = TOTAL_GRAINS-length
  
  ratio = int(length/more_needed)
  
  NEW_TYPE_PATH = os.path.join(NEW_DATA_PATH,grain_type)
  os.makedirs(NEW_TYPE_PATH)
  
      
  idx = 0
  
  for i in range(1,ratio+1):
    for gr in grains:
      gp = os.path.join(p, gr)
      img = cv2.imread(gp)
      (h,w) = img.shape[:2]
      centre = (w//2, h//2)
      
      img_name = "{}_{}.png".format(grain_type, idx)
      img_url = os.path.join(NEW_TYPE_PATH, img_name)
      idx+=1
      
      cv2.imwrite(img_url, img)
      
      M = cv2.getRotationMatrix2D(centre, 40*i, 1.0)
      rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
      
      img_name = "{}_{}.png".format(grain_type, idx)
      img_url = os.path.join(NEW_TYPE_PATH, img_name)
      idx+=1
      
      cv2.imwrite(img_url, rotated)
      

## Step 3: Prepare the data to feed to the Model

### Encoding the type of the grain

In [0]:
PATH = 'ENLARGED_TRAINABLE_DATASET'
label_encoder = {}
label_decoder = {}
i = 0
for grain_type in os.listdir(PATH):
  

  
  label_encoder[str(grain_type)] = i
  label_decoder[i] = grain_type
  i+=1
  
print(label_encoder)
print(label_decoder)

### Spliting the training and validation data.
We save the paths of the images to pass these to Custom Data Generator

In [0]:
# Partition to store all the image path location partiton{'train':[], 'validation':[]}
partition = {}
partition['train'] = []
partition['validation'] = []

# Label dict to store the path as key and label index as value
label = {}


for grain_type in os.listdir(PATH):
  #print("Collecting type {} has {} items".format(grain_type, len(os.listdir(PATH+'/'+grain_type))  ))


    
  type_path = os.path.join(PATH,grain_type)
  
  type_list = os.listdir(type_path)
  
  # We shuffle the image lists
  np.random.seed(10)
  
  fixed_type_list = np.random.choice(type_list, size=len(type_list), replace=False)
  
  train_list = fixed_type_list[3000:]
  validation_list = fixed_type_list[:3000]
  
  for item in train_list:
    item_path = os.path.join(type_path, item)
    partition['train'].append(item_path)
    label[item_path] = label_encoder[grain_type] 
    
  for item in validation_list:
    item_path = os.path.join(type_path, item)
    partition['validation'].append(item_path)
    label[item_path] = label_encoder[grain_type]
  
  

### Defining the Custom Keras Data Generator
As the size of the Dataset is large. So it is better to use Data Generator to avoid memory issue.
This [post](https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly) helped.

In [0]:
import numpy as np
import keras
import imutils

no_classes = 8
 
########## Custom Data Generator Class #################

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=64, dim=(128,128), n_channels=3,
                 n_classes=no_classes, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find .list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
         
      

    def __data_generation(self,list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            try:
              _ = (cv2.imread(ID).shape!=(128,128,3))
            except:
              continue
            try:
              X[i,] = cv2.imread(ID)*(1.0/255.0)
              #X_[i,] = generate_data(ID)
            except:
              continue
                
            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


### Creating the Custom Data Generator Object

In [0]:
import numpy as np

from keras.models import Sequential

training_generator = DataGenerator(partition['train'], label)
validation_generator = DataGenerator(partition['validation'], label)


## Step 4:  Defining our Keras Model

In [0]:
########### Conv Model #############

from keras.layers import *

model=Sequential()
input_shape=(128,128,3)
model = Sequential()

model.add(Conv2D(6, (5, 5), input_shape=input_shape, padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (5, 5),padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())


model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(no_classes, activation = 'softmax'))

model.summary()


model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

## Step 5: Training the Model

In [0]:

from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint('model-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', 
                             verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

callback_list = [checkpoint, earlyStopping] #



  
model.fit_generator(generator=training_generator,
                      epochs = 10,
                      validation_data=validation_generator,
                      use_multiprocessing=True,workers=200,
                      callbacks=callback_list
                      )

## Step 6: Making Predictions

In [0]:
def pred(image_path):
  
  k = np.expand_dims(cv2.imread(image_path)*(1.0/255.0), axis=0)
  p = model.predict(k)
  p = np.argmax(p)
  
  return label_decoder[p]


# Approach with Morphological Features
We also tested the result with Morphological Feature and different Machine Leanring Classifier. For this we have collected  Six morphological features. Using the following opencv functions we accomplish this. 

<img src="https://github.com/Mushahid2521/Rice-Grain-Purity-Analysis-Using-Deep-Learning/blob/master/images/ellipse.jpg?raw=1" alt="Drawing;"/>
<img src="https://github.com/Mushahid2521/Rice-Grain-Purity-Analysis-Using-Deep-Learning/blob/master/images/rectangle.jpg?raw=1" alt="Drawing" style="width: 200px;"/>

```fitEllipse()``` gives major axis length (majLen)  and minor axis length (minLen)

```minAreaRect()```  gives length (l) and width (w)

```cv2.contourArea``` gives area (A)

```cv2.arcLength``` gives perimmeter (P)






   
 **Six Features:**
1.   Area by Perimeter Ratio:   ($\frac{A}{P}$)
2.   Ratio to the Area to the total of area and perimeter:  $\frac{A}{(A+P)}$
3.   Aspect Ratio: ($\frac{majLen}{minlen}$)
4.   Retangularity: $\frac{A}{(l*w)}$ 
5.   Equivalent Diameter: $\sqrt{\frac{4A}{\pi}}$
6.   Shape factor 3: $\frac{A}{l*w}$

In [0]:
import numpy as np
import keras
import imutils
import math

no_classes = 8

def generate_data(ID):

    img = cv2.imread(ID)
    img_grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    canny_img = cv2.Canny(img_grey, 100, 100)
      
    cnts = cv2.findContours(canny_img.copy(),  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
      
    for c in cnts:
      rect = cv2.minAreaRect(c)
      ((_, _), (p,q), _) = rect
      if(p>=q):
          length = p
          width = q
      else:
          length = q
          width = p

      area = cv2.contourArea(cnts[0])
      perimeter = cv2.arcLength(cnts[0], closed=True)
      
      try:
        (_,_), (majax, minax), _ = cv2.fitEllipse(cnts[0])
        if(ax1>=ax2):
          majax = ax1
          minax = ax2
        else:
          majax = ax2
          minax = ax1
        return [area/perimeter, area/(area+perimeter), maxax/minax, area/(length*width), math.sqrt(4*area/math.pi), area/(majax*minax)]
      
      except:
        # Sometimes the Fit Ellipse function throws error. So we handle it by try catch block
        return [area/perimeter, area/(area+perimeter), length/width, area/(length*width), math.sqrt(4*area/math.pi), area/(length*width)]
        

In [0]:
x_train = []
y_train = []
x_test = []
y_test = []

PATH = 'ENLARGED_TRAINABLE_DATASET'

for grain_type in [os.listdir(PATH)[7]]:
  
  print(f"Collecting type {grain_type}")
    
  type_path = os.path.join(PATH,grain_type)
  
  type_list = os.listdir(type_path)
 
  np.random.seed(10)
  fixed_type_list = np.random.choice(type_list, size=len(type_list), replace=False)

  i = 0 
  for item in fixed_type_list:
    
    item_path = os.path.join(type_path, item)
    try:
      dat = generate_data(item_path)
      i+=1
    except:
      continue
      
    if(i%1000==0):
      print(i) 
    
    # We keep 3000 for test and rest for training 
    if(i<=3000):
      x_test.append(dat)
      y_test.append(label_encoder[grain_type])
    #print("Done with Validation")
    
    else:
      x_train.append(dat)
      y_train.append(label_encoder[grain_type])
      


### KNN Classifier:

In [0]:
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import confusion_matrix, classification_report

knn = KNeighborsClassifier(n_neighbors = 3).fit(x_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(x_test, y_test) 
print(accuracy) 
  
# creating a confusion matrix 
knn_predictions = knn.predict(x_test)  
cm = confusion_matrix(y_test, knn_predictions) 
target_names = label_encoder.keys()
print(classification_report(y_test, knn_predictions, target_names=target_names))
print(cm)

## Decision Tree Classifier

In [0]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth=10).fit(x_train, y_train) 
dtree_predictions = dtree_model.predict(x_test) 

# accuracy on X_test
acc = dtree_model.score(x_test, y_test)
print(f"Accuracy: {acc}")


# creating a confusion matrix 
print(confusion_matrix(y_test,dtree_predictions))
print('Classification Report')
target_names = label_encoder.keys()
print(classification_report(y_test, dtree_predictions, target_names=target_names))

## Neural Network

In [0]:
from keras.layers import Dense, Input
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

model = Sequential()
model.add(Dense(140, input_shape=(6,), activation='relu'))
model.add(Dense(140, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_tr = to_categorical(y_train)
model.fit(x_train, y_tr, epochs=6, batch_size=64, validation_data=(x_test,to_categorical(y_test)))


preds = model.predict_classes(x_test)
cm = confusion_matrix(y_test, preds) 

target_names = label_encoder.keys()
print(classification_report(y_test, preds, target_names=target_names))